# Fesmaro 2025 Analisis Big Data  


## LDA & LSA

In [24]:
import re
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LsiModel

In [2]:
import gdown 
url = "https://drive.google.com/drive/u/4/folders/1La029vITSdOyDC-TaKZ9kxVk0twOneFv"
gdown.download_folder(url)

Retrieving folder contents


Processing file 14XpIdfj6FN9ckvBVrRp9lUa-OtuDIu5O df_processed.csv
Processing file 1ZwRcWT9RubqpZ_qFukgq9ZyO3Ob0I0Q9 df_test.csv
Processing file 1Btr1jmjO2fx8Uh6_4-It79Mf5pLdXQcy df_train.csv
Processing file 1giX5L2NXjxDXepK3wsaRNJjUcSzZf_9P df_val.csv
Processing file 1YioeFhGr0yUFVX3dFcIwzFsPFfd11LKj final_df.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=14XpIdfj6FN9ckvBVrRp9lUa-OtuDIu5O
From (redirected): https://drive.google.com/uc?id=14XpIdfj6FN9ckvBVrRp9lUa-OtuDIu5O&confirm=t&uuid=a155c2dc-d2af-4c6f-92ff-bfb1faee881a
To: /kaggle/working/data/df_processed.csv
100%|██████████| 128M/128M [00:01<00:00, 82.5MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1ZwRcWT9RubqpZ_qFukgq9ZyO3Ob0I0Q9
To: /kaggle/working/data/df_test.csv
100%|██████████| 12.1M/12.1M [00:00<00:00, 33.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Btr1jmjO2fx8Uh6_4-It79Mf5pLdXQcy
To: /kaggle/working/data/df_train.csv
100%|██████████| 40.8M/40.8M [00:00<00:00, 62.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1giX5L2NXjxDXepK3wsaRNJjUcSzZf_9P
To: /kaggle/working/data/df_val.csv
100%|██████████| 8.06M/8.06M [00:00<00:00, 32.0MB/s]
Downloading...
From: https://drive.googl

['/kaggle/working/data/df_processed.csv',
 '/kaggle/working/data/df_test.csv',
 '/kaggle/working/data/df_train.csv',
 '/kaggle/working/data/df_val.csv',
 '/kaggle/working/data/final_df.csv']

In [3]:
!ls /kaggle/working/data

df_processed.csv  df_test.csv  df_train.csv  df_val.csv  final_df.csv


In [18]:
df = pd.read_csv("/kaggle/working/data/final_df.csv")

In [19]:
df.head()

,label,lemmatized_text,text_length,word_count,avg_word_length,exclamation_count,question_count,uppercase_ratio,positive_word_count,negative_word_count,sentiment_word_ratio
0,1,use book product herb easy access material giv...,357,63,4.682540,0,0,0.000000,1,0,2.0
1,1,cd great example heavy hardcore softer edge no...,552,100,4.530000,0,0,0.009901,2,0,3.0
2,0,not read book yet review sample kindle point s...,373,74,4.054054,1,0,0.000000,0,0,1.0
3,1,movie one favorite partly michigan favorite so...,558,105,4.323810,1,0,0.000000,3,0,4.0
4,1,fun ten barrel monkey movie escapist product s...,391,63,5.222222,2,0,0.000000,1,0,2.0


In [20]:
# Tokenize the lemmatized text
# This step assumes 'lemmatized_text' contains strings of space-separated words.
# If it already contains lists of tokens, you can skip this.
tokenized_texts = [word_tokenize(text.lower()) for text in df['lemmatized_text']]

In [21]:
# --- 3. Create Gensim Dictionary and Corpus ---
# Create a dictionary representation of the documents.
dictionary = Dictionary(tokenized_texts)

# **Optional but Recommended:** Filter out extreme words
# Filter out words that appear in fewer than (e.g.) 3 documents
# or more than 50% of the documents. Adjust these values based on your dataset size.
no_below = min(3, len(tokenized_texts) // 2) # Ensure no_below isn't too high for small samples
print(f"Filtering dictionary: keep tokens appearing in at least {no_below} documents and less than 50% of documents.")
dictionary.filter_extremes(no_below=no_below, no_above=0.5)

# Create a Corpus: Convert tokenized documents into bag-of-words vectors.
# This is a list where each element corresponds to a document,
# represented as a list of (word_id, word_frequency) tuples.
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

# Check if the corpus is empty after filtering (can happen with small/similar samples)
if not corpus or all(not doc for doc in corpus):
    print("\nWarning: The corpus is empty or contains only empty documents after filtering.")
    print("This might be due to strict filtering or very short/similar input texts.")
    print("Consider adjusting filter_extremes parameters (no_below, no_above) or checking your data.")
    # Exit or handle this case appropriately
    exit()
elif not dictionary:
     print("\nWarning: The dictionary is empty after filtering.")
     print("This might be due to strict filtering or very short/similar input texts.")
     print("Consider adjusting filter_extremes parameters (no_below, no_above) or checking your data.")
     # Exit or handle this case appropriately
     exit()


print(f"\nNumber of unique tokens after filtering: {len(dictionary)}")
print(f"Number of documents: {len(corpus)}")

Filtering dictionary: keep tokens appearing in at least 3 documents and less than 50% of documents.

Number of unique tokens after filtering: 29627
Number of documents: 100000


In [22]:
# --- 4. Build LDA Model ---
print("\n--- Building LDA Model ---")
# **Important Hyperparameter:** num_topics
# You need to choose the number of topics you want to extract.
# This often requires experimentation and evaluation (e.g., using coherence scores).
# Let's start with 3 topics for this small example. For 100k documents, you'd likely use more (e.g., 20, 50, 100+).
num_topics_lda = 3

# Train the LDA model
# `passes`: Number of passes through the corpus during training. More passes can lead to better topics but take longer.
# `random_state`: For reproducibility.
try:
    lda_model = LdaModel(corpus=corpus,
                         id2word=dictionary,
                         num_topics=num_topics_lda,
                         random_state=42,
                         passes=10, # Increase passes for potentially better results on larger data
                         # alpha='auto', # Let Gensim learn asymmetric alpha from data (optional)
                         # eta='auto'    # Let Gensim learn asymmetric eta from data (optional)
                        )

    # Print the topics found by the LDA model
    print(f"\nLDA Topics (Top words per topic):")
    # `num_words`: How many keywords to display for each topic
    topics_lda = lda_model.print_topics(num_words=5)
    for topic_num, topic_words in topics_lda:
        print(f"Topic {topic_num}: {topic_words}")

except ValueError as e:
    print(f"\nError building LDA model: {e}")
    print("This might happen if the corpus is effectively empty after filtering.")


--- Building LDA Model ---

LDA Topics (Top words per topic):
Topic 0: 0.039*"number" + 0.011*"one" + 0.010*"would" + 0.009*"great" + 0.009*"work"
Topic 1: 0.049*"book" + 0.015*"read" + 0.010*"one" + 0.010*"movie" + 0.009*"story"
Topic 2: 0.027*"number" + 0.016*"cd" + 0.014*"like" + 0.013*"album" + 0.013*"one"


In [25]:
# --- 5. Build LSA (LSI) Model ---
print("\n--- Building LSA (LSI) Model ---")
# **Important Hyperparameter:** num_topics
# Similar to LDA, you choose the number of desired dimensions/topics.
# LSI often benefits from starting with a larger number of topics initially (e.g., 100-500)
# and then potentially using only the top ones, but let's stick with a small number for illustration.
num_topics_lsa = 3

# Train the LSI model
# Note: LSI often works well with TF-IDF input, but we'll use the BoW corpus here for direct comparison with LDA setup.
# For larger datasets, consider creating a TF-IDF corpus first:
# from gensim.models import TfidfModel
# tfidf = TfidfModel(corpus)
# corpus_tfidf = tfidf[corpus]
# lsi_model = LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=num_topics_lsa)
try:
    lsi_model = LsiModel(corpus=corpus,
                         id2word=dictionary,
                         num_topics=num_topics_lsa
                         # decay=1.0, # Default value, controls diminishing influence of later documents in online training
                         # onepass=True # Set to False for multi-pass SVD, potentially more accurate but slower/memory intensive
                        )

    # Print the topics found by the LSI model
    print(f"\nLSA/LSI Topics (Top words per topic):")
    # `num_words`: How many keywords to display for each topic
    topics_lsa = lsi_model.print_topics(num_words=5)
    for topic_num, topic_words in topics_lsa:
        # Clean up the output format which often looks like '0.700*"word1" + 0.500*"word2" ...'
        cleaned_topic = re.sub(r'-?\d+\.\d+\*', '', topic_words).replace('"', '').replace(' + ', ', ')
        print(f"Topic {topic_num}: {cleaned_topic.strip()}")

except Exception as e: # LSI can raise different errors, e.g., related to SVD
    print(f"\nError building LSI model: {e}")
    print("This might happen with very small or sparse corpora.")



--- Building LSA (LSI) Model ---

LSA/LSI Topics (Top words per topic):
Topic 0: number, book, one, great, would
Topic 1: book, number, read, one, good
Topic 2: book, one, number, movie, like
